In [6]:
import ipywidgets as widgets
from ipywidgets import Checkbox, VBox, HBox
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

df = vaex.open("../hdf5_pinIO/test8/pin_out.h5")
print(df.ADDRESS.count())
namesFromFile = ["Hardcoded Tag 1", "2", "Three", "4444"]
tagsFromFile = np.unique(df.tag.values).tolist()
numTags = len(namesFromFile)

df['index'] = np.arange(0, df.ADDRESS.count())
df.select(True, name='structures')
df.select(True, name='rw')
df.select(True, name='total')

for tag in tagsFromFile:
    df.select(df.tag == tag, mode='or')


def updateGraph(change):
    if(change.name == 'value'):
        
        name = change.owner.description
        tag = tagsFromFile[namesFromFile.index(name)]

        if(change.new == True):
            df.select(df.tag == tag, mode='or', name='structures')
            df.select("structures", mode='replace', name='total')
            df.select("rw", mode='and', name='total')
        else:
            df.select(df.tag == tag, mode='subtract', name='structures')
            df.select("structures", mode='replace', name='total')
            df.select("rw", mode='and', name='total')
    df.select('total')
        
def updateRead(change):
    if(change.name == 'value'):
        if(change.new == True):
            df.select(df.WRITE == 0, mode='or', name='rw')
            df.select("structures", mode='replace', name='total')
            df.select("rw", mode='and', name='total')
        else:
            df.select(df.WRITE == 0, mode='subtract', name='rw')
            df.select("structures", mode='replace', name='total')
            df.select("rw", mode='and', name='total')
    df.select('total')
            
def updateWrite(change):
    if(change.name == 'value'):
        if(change.new == True):
            df.select(df.WRITE == 1, mode='or', name='rw')
            df.select("structures", mode='replace', name='total')
            df.select("rw", mode='and', name='total')
        else:
            df.select(df.WRITE == 1, mode='subtract', name='rw')
            df.select("structures", mode='replace', name='total')
            df.select("rw", mode='and', name='total')
    df.select('total')
            
tagButtons = [Checkbox(description=namesFromFile[i], value=True, disabled=False, indent=False) for i in range(numTags)]


    
rwButtons = [Checkbox(description="Reads", value=True, disabled=False, indent=False),
             Checkbox(description="Writes", value=True, disabled=False, indent=False)]

hitMissButton = [Checkbox(description="Hits", value=True, disabled=False, indent=False),
                 Checkbox(description="Misses", value=True, disabled=False, indent=False)]
    
    
for i in range(numTags):
    tagButtons[i].observe(updateGraph)
    
rwButtons[0].observe(updateRead)
rwButtons[1].observe(updateWrite)
    
checks = HBox([VBox(tagButtons), VBox(rwButtons), VBox(hitMissButton)])

df.plot_widget(df.index, df.ADDRESS, selection=[True], backend='bqplot', tool_select=True)


display(checks)

1000000


PlotTemplate(components={'main-widget': VBox(children=(VBox(children=(Figure(axes=[Axis(color='#666', grid_col…

In [4]:
import ipywidgets as widgets
import inspect
from ipywidgets import Checkbox, VBox
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt



df = vaex.open("../hdf5_pinIO/test8/pin_out.h5")
df

#,ADDRESS,WRITE,tag
<i style='opacity: 0.6'>0</i>,0,0,0
<i style='opacity: 0.6'>1</i>,1,1,1
<i style='opacity: 0.6'>2</i>,2,0,2
<i style='opacity: 0.6'>3</i>,3,1,3
<i style='opacity: 0.6'>4</i>,4,0,0
...,...,...,...
"<i style='opacity: 0.6'>999,995</i>",999995,1,3
"<i style='opacity: 0.6'>999,996</i>",999996,0,0
"<i style='opacity: 0.6'>999,997</i>",999997,1,1
"<i style='opacity: 0.6'>999,998</i>",999998,0,2
